# Contrôle continu n°2 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, KFold

prices = pd.read_csv("../data/price_availability.csv", sep=";")
listings = pd.read_csv("../data/listings_final.csv", sep=";")
intersects = np.intersect1d(np.unique(prices['listing_id']), np.unique(listings['listing_id']))
listings_prices = prices[prices['listing_id'].isin(intersects)]

# Plus le droit d'utiliser ces variables !
del prices
del intersects

dates =  pd.to_datetime(listings_prices["day"], format='%Y-%m-%d')
range_prices = listings_prices.assign(date=dates).set_index(dates).groupby(['listing_id', pd.Grouper(key='date', freq='7D')]).mean().reset_index()

print("Chargement des données effectué.")

Chargement des données effectué.


## (•) Question 1

**Récupérer la colône type du dataset listings***.

0    private_room
1     entire_home
2    private_room
3     entire_home
4     entire_home
Name: type, dtype: object


## (•) Question 2

**Supprimer la ligne numéro 547 du dataset listings***.

## (•) Question 3

**Construire un nouveau dataset sans le type private_room à partir du dataset listings***.

## (•) Question 4

**Effectuer une encodage par dictionnaire sur la date dans une nouvelle colonne nommée *date_dict***.

,listing_id,date,available,local_price,min_nights,date_dict
0,56093,2018-08-27,0.0,170.0,4.0,0
1,56093,2018-09-03,0.0,170.0,4.0,1
2,56093,2018-09-10,0.0,170.0,4.0,2
3,56093,2018-09-17,0.0,170.0,4.0,3
4,56093,2018-09-24,0.0,170.0,4.0,4


## (•) Question 5

**Stocker dans la variable *one_hot_dates* l'encodage one-hot des dates.**

/Users/maxime/Documents/Blent_Formations/DataGramxAirBnB/venv/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(18981, 19)

## (••) Question 6

**Ajouter la matrice de vecteurs one-hot au DataFrame *range_prices_nights* et stocker dans un nouveau DataFrame nommé *dataset*.**

,listing_id,date,available,local_price,min_nights,date_dict,name,type,city,neighborhood,...,oh_10,oh_11,oh_12,oh_13,oh_14,oh_15,oh_16,oh_17,oh_18,oh_19
0,56093,2018-08-27,0.0,170.0,4.0,0,Beau duplex dans le Marais,entire_home,Paris,3e arrondissement,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,56093,2018-09-03,0.0,170.0,4.0,1,Beau duplex dans le Marais,entire_home,Paris,3e arrondissement,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,56093,2018-09-10,0.0,170.0,4.0,2,Beau duplex dans le Marais,entire_home,Paris,3e arrondissement,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,56093,2018-09-17,0.0,170.0,4.0,3,Beau duplex dans le Marais,entire_home,Paris,3e arrondissement,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,56093,2018-09-24,0.0,170.0,4.0,4,Beau duplex dans le Marais,entire_home,Paris,3e arrondissement,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#le résultat 
dataset.columns

Index(['listing_id', 'date', 'available', 'local_price', 'min_nights',
       'date_dict', 'name', 'type', 'city', 'neighborhood', 'latitude',
       'longitude', 'person_capacity', 'beds', 'bedrooms', 'bathrooms',
       'is_rebookable', 'is_new_listing', 'is_fully_refundable',
       'is_host_highly_rated', 'is_business_travel_ready',
       'pricing_weekly_factor', 'pricing_monthly_factor', 'oh_1', 'oh_2',
       'oh_3', 'oh_4', 'oh_5', 'oh_6', 'oh_7', 'oh_8', 'oh_9', 'oh_10',
       'oh_11', 'oh_12', 'oh_13', 'oh_14', 'oh_15', 'oh_16', 'oh_17', 'oh_18',
       'oh_19'],
      dtype='object')

## (••) Question 7

**Entrainer un modèle de Gradient Boosting avec les variables suivantes :**

- Les coordonnées géographiques
- La capacité, le nombre de chambres et de salles de bains
- La date encodé sous format one-hot

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(
    dataset.iloc[:, [10, 11, 12, 14, 15 ] + [ i for i in range(len(dataset.columns) - 19, len(dataset.columns)) ]],
    dataset.loc[:, "local_price"],
    test_size=0.2)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=200, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=1, subsample=1)

## (•) Question 8

**Calculer le score $R^2$ du modèle.**

Variance score: 0.7597


## (•) Question 9

**Tuner les hyper-paramètres suivants en utilisant les valeurs de votre choix :**

- La profondeur *max_depth*
- Le nombre d'arbres *n_estimators*
- Le taux d'apprentissage *learning_rate*

**Attention**, cela peut prendre jusqu'à plusieurs minutes.

In [36]:
from sklearn.model_selection import GridSearchCV

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/Users/maxime/Documents/Blent_Formations/DataGramxAirBnB/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   31.5s
[Parallel(n_jobs=5)]: Done  81 out of  81 | elapsed:  1.8min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=5,
       param_grid={'max_depth': [2, 3, 4], 'learning_rate': [0.5, 1.0, 1.5], 'n_estimators': [100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=3)

Score : 79.15%
Paramètres : {'learning_rate': 0.5, 'max_depth': 2, 'n_estimators': 100}
Score : 86.27%
Paramètres : {'learning_rate': 0.5, 'max_depth': 2, 'n_estimators': 200}
Score : 89.28%
Paramètres : {'learning_rate': 0.5, 'max_depth': 2, 'n_estimators': 300}
Score : 88.62%
Paramètres : {'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 100}
Score : 92.97%
Paramètres : {'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 200}
Score : 94.15%
Paramètres : {'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 300}
Score : 92.95%
Paramètres : {'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 100}
Score : 94.82%
Paramètres : {'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 200}
Score : 95.04%
Paramètres : {'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 300}
Score : 82.73%
Paramètres : {'learning_rate': 1.0, 'max_depth': 2, 'n_estimators': 100}
Score : 89.46%
Paramètres : {'learning_rate': 1.0, 'max_depth': 2, 'n_estimators': 200}
Score : 92.11%
Paramètres : {'le

## (••) Question 10

**Etudier la stabilité du modèle en déterminant un intervalle de confiance de niveau 95\% en utilisant les paramètres optimaux déterminés précedemment.**

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.5, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

Intervalle de confiance de niveau 95% : 94.7% ± 0.016%


Comparer avec les résultats obtenus lorsque l'on avait réalisé l'encodage de la date uniquement par dictionnaire.